In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier\\research'

In [2]:
import os

os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    data_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.SMSClassifier.constants import CONFIG_PATH
from src.SMSClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_file= CONFIG_PATH):
        self.config_path = read_yaml(config_file)
        
        create_directories([self.config_path.artifacts_root])
        
    
    def dataIngestionConfig(self) -> DataIngestionConfig:
        config = self.config_path.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            data_url = config.data_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )   
        
        return data_ingestion_config  

In [6]:
import os
import urllib.request as request
from src.SMSClassifier.logging import logger
from src.SMSClassifier.utils.common import get_size
import zipfile

In [7]:
class DataIngestion:
    def __init__(self, confg: DataIngestionConfig):
        self.config = confg
        
    
    def download_dataset(self):
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                                    url= self.config.data_url,
                                    filename=self.config.local_data_file
                                )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exist, size--> {get_size(Path(self.config.local_data_file))}")
    
    
    def extract_zip(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.dataIngestionConfig()
    data_ingestinion = DataIngestion(confg=data_ingestion_config)
    data_ingestinion.download_dataset()
    data_ingestinion.extract_zip()
except Exception as e:
    raise e

[2024-01-29 08:51:49,238: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-29 08:51:49,238: INFO: common: created directory at: artifacts]
[2024-01-29 08:51:49,243: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-29 08:51:50,429: INFO: 597450681: artifacts/data_ingestion/SMS-Spam.zip download! with following info: 
Connection: close
Content-Length: 216360
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "66dc368a6dcf0fb56b756ac7235de703990cb9de077618af577feb42c464ef95"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8074:139773:F0042:F7B56:65B76723
Accept-Ranges: bytes
Date: Mon, 29 Jan 2024 08:51:47 GMT
Via: 1.1 varnish
X-Served-By: cache-lcy-eglc8600020-LCY
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1706518308.643506,VS0,VE183
Vary: Authorization